In [1]:
! git clone https://github.com/jimmyjbling/SGC-DEL-ML-WDR91.git
! git lfs install --skip-smudge
! git clone https://github.com/NikSchap-2107/Target2035_Aircheck_Utils.git
! git lfs pull
! git lfs install --force
! pip install rdkit-pypi
! pip install rdkit

Cloning into 'SGC-DEL-ML-WDR91'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 23 (delta 6), reused 23 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 6.03 MiB | 7.20 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Git LFS initialized.
Cloning into 'Target2035_Aircheck_Utils'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 119 (delta 49), reused 104 (delta 39), pack-reused 0 (from 0)
Receiving objects: 100% (119/119), 86.54 KiB | 3.46 MiB/s, done.
Resolving deltas: 100% (49/49), done.
Not in a git repository.
Git LFS initialized.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 13.0 MB/s eta 0:00:00


In [ ]:
import sys
sys.path.append("/content/Target2035_Aircheck_Utils/ReadingParquetFiles")
sys.path.append("/content/SGC-DEL-ML-WDR91/scripts")
from read_parquet_utils import read_parquet_file, process_column_to_array
from sklearn.model_selection import train_test_split
import numpy as np
from rdkit.SimDivFilters import rdSimDivPickers
from rdkit import DataStructs
from tqdm import tqdm
from collections import defaultdict

data_file = "/content/Target2035_Aircheck_Utils/Data/WDR91.parquet"
df = read_parquet_file(data_file, columns = ['ECFP6', 'LABEL']) #bECFP6 for initial model ; bECFP6-ECFP6-atompairs for advanced model ; available: ECFP4, ECFP6, FCFP4, FCFP6, TOPTOR, MACCS, RDK, AVALON

FileNotFoundError: [Errno 2] No such file or directory: '/content/Target2035_Aircheck_Utils/Data/WDR91.parquet'

In [ ]:
df.head()

,ECFP6,LABEL
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
3,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
4,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",0


In [ ]:
df.value_counts('LABEL')

,count
LABEL,
0,346817
1,28778


In [ ]:
fps = process_column_to_array(df, 'ECFP6')
labels = np.array(df['LABEL'].tolist())
print(fps)
print(labels)
print(fps.max())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]
[0 0 0 ... 0 0 0]
24


In [ ]:
def numpy_2_fp(array):
    fp = DataStructs.cDataStructs.UIntSparseIntVect(len(array))
    for ix, value in enumerate(array):
        fp[ix] = int(value)
    return fp

def distij(i, j, fps=fps):
  return 1 - DataStructs.DiceSimilarity(fps[i], fps[j])

def assignPointsToClusters(picks, fps):
    clusters = defaultdict(list)
    for i, idx in enumerate(picks):
        clusters[i].append(idx)
    sims = np.zeros((len(picks), len(fps)))
    for i in tqdm(range(len(picks))):
        pick = picks[i]
        sims[i, :] = DataStructs.BulkTanimotoSimilarity(fps[pick], fps)
        sims[i, i] = 0
    best = np.argmax(sims, axis=0)
    for i, idx in enumerate(best):
        if i not in picks:
            clusters[idx].append(i)
    return clusters

In [ ]:
fps = [numpy_2_fp(fp) for fp in fps]

In [ ]:
lp = rdSimDivPickers.LeaderPicker()
thresh = 0.65  # <- minimum distance between cluster centroids
picks = lp.LazyPick(distij, len(fps), thresh)
clusters = assignPointsToClusters(picks, fps)
cluster_ids = np.zero(len(fps))
for key, val in clusters.items():
    cluster_ids[val] = key